In [1]:
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout, TimeDistributed, Reshape, Lambda, Input
from keras.layers import LSTM, Bidirectional
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.regularizers import l1
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
import pretty_midi

import nbimporter
import import_ipynb
import Preprocess, Midiz
import glob
import matplotlib.pyplot as plt
import numpy as np
import os, io
import random
pretty_midi.pretty_midi.MAX_TICK = 1e30 # Prevents Maximum tick overflow for the midi tracks that are big (normal songs).
np.set_printoptions(threshold=np.nan) # print shows all.
from IPython.core.interactiveshell import InteractiveShell
np.set_printoptions(threshold=12000)

c:\program files\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Importing Jupyter notebook from Preprocess.ipynb
Importing Jupyter notebook from Midiz.ipynb


In [2]:
def training_generator(window):
    path = glob.glob('G:\ΣΧΟΛΗ p12127\Πτυχιακη\MAPS Dataset\Music Dataset\Train Dataset\**\**\**\*.wav')
    
    while True:
        selected_file = random.choice(path) #this selects a random file anew.
        with open((selected_file),"r") as f:
            wave_path = str(f)
            wave_path = wave_path[25:].split("' mode='r'")[0]
            midi_path = wave_path.split('.wav')[0] + '.mid'
            
            fbanks = Preprocess.process(wave_path, small_window_size=window)
            fbanks = fbanks.T
            midi_labels = Midiz.piano_roll(midi_path, fbanks.shape[0])
            
            #get a random start, but 50 frames less than max, because the sample is 50 frames length.
            Xtrain = np.empty((32,50,88))
            Ytrain = np.empty((32,88))
            
            # get a random number out of the frames of filterbanks, but 50 frames less than max.
            random_index = random.randint(0,fbanks.shape[0]-50)
            
            #print("index chosen: ", random_index)
            #print(fbanks[random_index:random_index+50,:], "\n\n LABELS: \n\n")
            #print(midi_labels[random_index:random_index+50,:])
            
            for batch in range(32):
                for frame in range(random_index,random_index+50,25):
                    Ytrain[batch] = midi_labels[frame+25]
                    for i in range(50):
                        Xtrain[batch,i] = fbanks[random_index+i]

            yield Xtrain, Ytrain

In [3]:
def validation_generator(window):
    path = glob.iglob('G:\ΣΧΟΛΗ p12127\Πτυχιακη\MAPS Dataset\Music Dataset\Validation Dataset\**\**\**\*.wav')
    
    while True:
        selected_file = next(path)
        #print(selected_file)
        with open((selected_file),"r") as f:
            wave_path = str(f)
            wave_path = wave_path[25:].split("' mode='r'")[0]
            midi_path = wave_path.split('.wav')[0] + '.mid'
            Xvalid = np.empty((32,50,88))
            Yvalid = np.empty((32,88))
            fbanks = Preprocess.process(wave_path, small_window_size=window)
            fbanks = fbanks.T
            midi_labels = Midiz.piano_roll(midi_path,fbanks.shape[0])
            batches = int(fbanks.shape[0]/(25*32)) # 32batch size, 50 frames BUT we overlap by 25. So slide=25.
            #print(batches, "Batches available.", fbanks.shape[0], "Samples")
            for iters in range(batches):
                for b in range(32):
                    try:
                        Yvalid[b,:] = midi_labels[(b+1)*25 + 25*iters]
                        #print("Yvalid labels: ", b*25+25*iters)
                    except IndexError:
                        Yvalid[b,:] = midi_labels[-1]
                    for i in range(50):
                        try:
                            Xvalid[b,i]=fbanks[i + 25*b + 32*25*iters]#iterate mult with 25 because of overlapping windows!
                        except IndexError:
                            Xvalid[b,i]=fbanks[-1]
                        #if i==0 :print(i + 25*b + 32*25*iters, b, iters )
            yield Xvalid,Yvalid

In [4]:
def note_model():
    batch_size, frames, banks = 32 ,50, 88
    model = Sequential()
    model.add(Bidirectional(LSTM(100, stateful=False, return_sequences=True,  kernel_regularizer=l1(10**(-4)), dropout=0.5),
    batch_size=batch_size,batch_input_shape =(batch_size, frames, banks)))
    model.add(Bidirectional(LSTM(100, stateful=False, return_sequences=False,  kernel_regularizer=l1(10**(-4)), dropout=0.5)))
    model.add(Dense(88, activation='sigmoid'))
    model.summary()
    return model

In [5]:
def train_note_model(model,window):
    checkpoint_path = "12 Oct run\\2x100units-loss=BCE-Model-{epoch:02d}-{val_acc:.2f}.hdf5"
    model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy']) #mean_squared_error or binary_crossentropy
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=0, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=10, min_lr=0.001)
    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    model.fit_generator(training_generator(window=window), steps_per_epoch=100, 
                        callbacks=[monitor, reduce_lr, checkpoint],
                        validation_data= validation_generator(window=window), validation_steps=4,
                        verbose=1, epochs=10, shuffle=False)
    
model = note_model()
#model = load_model('20 Sept run\\100units-loss=MSE-Model-03-0.07.hdf5')
#train_note_model(model, window=True)

# Continue training with the big window of FFT
#train_note_model(model, window=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (32, 50, 200)             151200    
_________________________________________________________________
bidirectional_2 (Bidirection (32, 200)                 240800    
_________________________________________________________________
dense_1 (Dense)              (32, 88)                  17688     
Total params: 409,688
Trainable params: 409,688
Non-trainable params: 0
_________________________________________________________________


In [11]:
'''make function where prints Ypred/Ytruth in a for loop using the 32batches.'''
#def make_pred(inputfile):
    
model = load_model('12 Oct run\\2x100units-loss=BCE-Model-10-0.96.hdf5')
#inputfile = 'G:\ΣΧΟΛΗ p12127\Πτυχιακη\MAPS Dataset\Test Dataset\MAPS_AkPnBcht_1\AkPnBcht\ISOL\CH\MAPS_ISOL_CH0.1_F_AkPnBcht.wav'
    
    
    
    
gen= validation_generator(window=True)
x,y = next(gen)
#yhat = model.predict_classes(x, batch_size=32, verbose=2)

#def make_pred(inputfile,notes=3):
'''inputfile= file to predict.
notes = number of notes per frame to predict.'''
    #make function that creates the frames for the file. similar to the generators.
yhat = model.predict(x)
indx = np.argsort(yhat)
ylist=[]
for i in range(yhat.shape[0]):
    for j in range(yhat.shape[1]):
        if indx[i,j] <= 5: #notes
            #print(indx[i,j], "th note: ", j)
            ylist.append(j)
    print("frame ", i, "= ", ylist)
    ylist[:]=[]

frame  0 =  [0, 10, 13, 49, 56, 80]
frame  1 =  [0, 9, 13, 53, 54, 77]
frame  2 =  [0, 8, 13, 44, 55, 75]
frame  3 =  [1, 7, 9, 39, 58, 77]
frame  4 =  [0, 14, 20, 45, 66, 84]
frame  5 =  [2, 12, 19, 44, 62, 81]
frame  6 =  [1, 7, 15, 41, 61, 78]
frame  7 =  [1, 7, 18, 40, 60, 78]
frame  8 =  [2, 8, 15, 42, 57, 78]
frame  9 =  [2, 8, 20, 38, 58, 80]
frame  10 =  [1, 11, 19, 48, 49, 81]
frame  11 =  [1, 11, 17, 45, 49, 85]
frame  12 =  [0, 17, 19, 44, 53, 85]
frame  13 =  [1, 10, 14, 45, 55, 83]
frame  14 =  [1, 9, 12, 48, 54, 81]
frame  15 =  [2, 7, 16, 39, 62, 75]
frame  16 =  [2, 7, 15, 37, 61, 75]
frame  17 =  [1, 10, 11, 37, 60, 78]
frame  18 =  [4, 18, 23, 43, 56, 81]
frame  19 =  [1, 16, 17, 39, 56, 72]
frame  20 =  [1, 8, 13, 40, 57, 76]
frame  21 =  [1, 11, 13, 46, 53, 78]
frame  22 =  [2, 12, 16, 47, 51, 76]
frame  23 =  [2, 11, 13, 43, 56, 75]
frame  24 =  [3, 11, 12, 41, 56, 75]
frame  25 =  [2, 12, 14, 39, 57, 79]
frame  26 =  [1, 8, 11, 36, 60, 76]
frame  27 =  [5, 14, 20,

In [7]:
#train_note_model(model, window=False)
help(Preprocess)

Help on module Preprocess:

NAME
    Preprocess

FUNCTIONS
    create_frames(data, nfft_length, number_of_frames)
    
    extract(inputfile, small_window_size=True)
    
    firdif(fb)
    
    process(inputfile, small_window_size)
    
    spectrogram(frames, nfft_length, sr)
        Hertz vocabulary.
    
    wave_plot(filterbanks)
    
    zero_pad(filterbanks)

DATA
    interact = <ipywidgets.widgets.interaction._InteractFactory object>

FILE
    c:\users\gio\desktop\p12127\jupyter notebook home\ptuxiakh\preprocess.ipynb


